This is an exemplary application of the use of CSDPP by Wang et al. for heterogeneous generalized zero-shot domain adaptation.

In [1]:
import scipy.io
from collections import Counter
import numpy as pd
from sklearn.model_selection import train_test_split
from src.pyth.CDSPP import *
from src.pyth.preprocessing import *
from src.pyth.evaluating import *
import random

Here, the OfficeHome data set is chosen.

In [2]:
PATH_source = "datasets/OfficeHome/OfficeHome-RealWorld-resnet50-noft.mat"
PATH_target = "datasets/OfficeHome/OfficeHome-Art-vgg16-noft.mat"
source = scipy.io.loadmat(PATH_source)
target = scipy.io.loadmat(PATH_target)
X_s = source['resnet50_features'].reshape(source['resnet50_features'].shape[0], source['resnet50_features'].shape[1])
y_s = source['labels'][0]
X_t = target['vgg16_features']
y_t = target['labels'][0]
X_train, X_test, y_train, y_test = train_test_split(X_t, y_t, test_size=0.5)

30 random classes are masked from the target and used for zero-shot recognition. The CDSPP class expects data in a (features, sample) format!

In [6]:
unknown = random.sample(range(65), 30)
X_seen, X_unseen, y_seen, y_unseen = split_masked_cells(X_train, y_train, unknown)
model = CDSPP(X_s.T, y_s, 500, 65, unknown)
model.fit_semi_supervised(X_seen.T, X_test.T, y_seen, part=0.75)
pred = model.predict(X_test.T)
res = h_score(y_test, pred, unknown)
print("H: "+str(res[0])+", Acc known: "+str(res[1])+", Acc unknwon: "+str(res[2]))    

H: 0.4720642206921562, Acc known: 0.6300832105832553, Acc unknwon: 0.3774126497758397
